# Climate Data Preparation

In [1]:
# Adjust Notebook Display
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
# Imports
import pandas as pd
import numpy as np
import os

In [3]:
# Set Repetitive Functions
def getLTA(dataset, variable, aggregationfunction, suffix):
    dataset['year'] = dataset['year'].astype(int)
    df = dataset[dataset['year'] <= 2017]
    LTA = pd.pivot_table(df,
                         values=[variable],
                         columns=['NUTS_ID'],
                         aggfunc=[aggregationfunction] )
    LTA = LTA.T
    LTA = LTA.droplevel(0)
    LTA = LTA.add_suffix(suffix)
    return LTA

## FAPAR

In [4]:
# Get Files in FAPAR Folder
location = "C:/Users/spiterisr/OneDrive - centralbankmalta.org/Working Papers/Beat the Heat Hackathon/Data/Climate Data/FAPAR/"
all_files = os.listdir(location)

In [5]:
# Store Files as 1 Dataframe

all_dfs = []

for file in all_files:
    if file.endswith("csv"):
        file_name = file.split("/")[-1]
        year = file_name.split("_")[-1].split(".")[0]
        
        fapar_yearly = pd.read_csv(location+file)
        fapar_yearly["year"] = year
        # column names has whitespaces around
        fapar_yearly.rename(columns=lambda x: x.strip(), inplace=True)
        all_dfs.append(fapar_yearly)
        
fapar = pd.concat(all_dfs)
fapar.dropna(subset=['MEDIAN'], inplace=True)
fapar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176330 entries, 0 to 17214
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   NUTS_ID    176330 non-null  object 
 1   TIMESTAMP  176330 non-null  object 
 2   MEDIAN     176330 non-null  float64
 3   year       176330 non-null  object 
dtypes: float64(1), object(3)
memory usage: 6.7+ MB


In [6]:
# Descriptive Statistics Format

fapar_aggregated = fapar.copy()

# calculate standard deviation per year
additional_variable = fapar_aggregated.groupby(['NUTS_ID', 'year']).agg({'MEDIAN':'std'})
additional_variable = additional_variable.add_suffix('_std')
fapar_aggregated = pd.merge(fapar_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate mean per year
additional_variable = fapar_aggregated.groupby(['NUTS_ID', 'year']).agg({'MEDIAN':'mean'})
additional_variable = additional_variable.add_suffix('_mean')
fapar_aggregated = pd.merge(fapar_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate min per year
additional_variable = fapar_aggregated.groupby(['NUTS_ID', 'year']).agg({'MEDIAN':'min'})
additional_variable = additional_variable.add_suffix('_min')
fapar_aggregated = pd.merge(fapar_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# sum months of low radiation activity
fapar_aggregated['fapar_low_months_to_remove'] = np.where(fapar_aggregated['MEDIAN'] < -1, 1, 0)
additional_variable = fapar_aggregated.groupby(['NUTS_ID', 'year']).agg({'fapar_low_months_to_remove':'sum'})
additional_variable = additional_variable.add_suffix('_new')
fapar_aggregated = pd.merge(fapar_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# rewrite column names
fapar_aggregated = fapar_aggregated.rename(columns={"fapar_low_months_to_remove_new": "fapar_low_months"})
fapar_aggregated = fapar_aggregated.drop(['MEDIAN', 'fapar_low_months_to_remove'], axis=1)
fapar_aggregated.columns = [str(col).replace("MEDIAN", "fapar")  for col in fapar_aggregated.columns]
# lets create yearly data. Just use any of  max or min function as it is already calculated for yearly data
f = {'fapar_std': 'min',
     'fapar_mean': 'min',
     'fapar_min': 'min', 
     'fapar_low_months': 'min'}
fapar_aggregated = fapar_aggregated.groupby(['NUTS_ID', 'year'], as_index=False).agg(f)

fapar_aggregated.head()

NUTS_ID  year  fapar_std  fapar_mean  fapar_min  fapar_low_months
0      AD  2012   0.816507    0.075833     -1.177                 1
1      AD  2013   0.889467   -0.151750     -1.400                 1
2      AD  2014   0.780246    0.313833     -0.972                 0
3      AD  2015   0.593244    0.697833     -0.420                 0
4      AD  2016   0.500969    0.948583      0.247                 0

In [7]:
# MIDAS Format

fapar['TIMESTAMP'] = pd.to_datetime(fapar['TIMESTAMP'])
fapar['month'] = fapar['TIMESTAMP'].dt.month

counter = 1
for i in fapar['month'].unique():
    if counter == 1:
        m_df = fapar[fapar['month'] == i]
        m_df = m_df[['NUTS_ID', 'year', 'MEDIAN']]
        m_df.rename(columns={'MEDIAN': 'FAPAR ['+str(i)+']'}, inplace=True)
    else:
        m2_df = fapar[fapar['month'] == i]
        m2_df = m2_df[['NUTS_ID', 'year', 'MEDIAN']]
        m2_df.rename(columns={'MEDIAN': 'FAPAR ['+str(i)+']'}, inplace=True)
        m_df = pd.merge(m_df, m2_df, on=['NUTS_ID', 'year'], how='outer')
    counter += 1

fapar_midas = m_df.copy()
fapar_midas.head()

NUTS_ID  year  FAPAR [1]  FAPAR [2]  FAPAR [3]  FAPAR [4]  FAPAR [5]  \
0      AD  2012      1.000      1.146      0.778     -0.020     -0.589   
1   AL011  2012     -0.509     -0.881     -1.102     -0.848     -0.167   
2   AL012  2012     -0.754     -1.022     -0.934     -0.741      0.526   
3   AL013  2012     -0.674     -1.356     -1.279     -0.998     -0.163   
4   AL014  2012     -0.355     -0.758     -0.201     -0.780      0.032   

   FAPAR [6]  FAPAR [7]  FAPAR [8]  FAPAR [9]  FAPAR [10]  FAPAR [11]  \
0      0.980      0.790     -0.484     -1.177      -0.636      -0.172   
1      0.788      0.616     -0.135     -0.924      -0.893       0.039   
2      0.246     -0.367     -0.745     -1.001      -1.278      -0.472   
3      0.949      0.208     -0.623     -0.986      -0.086       0.293   
4      0.351     -0.149     -0.663     -0.721      -0.625      -0.369   

   FAPAR [12]  
0      -0.706  
1      -0.035  
2       0.076  
3      -0.278  
4      -0.406

In [8]:
# Add LTAs

LTA = getLTA(fapar, 'MEDIAN', np.mean, '_LTA')
fapar_aggregated = pd.merge(fapar_aggregated, LTA, on=['NUTS_ID'])
fapar_aggregated.rename(columns={'MEDIAN_LTA': 'fapar_lta'}, inplace=True)
fapar_midas = pd.merge(fapar_midas, LTA, on=['NUTS_ID'])
fapar_midas.rename(columns={'MEDIAN_LTA': 'fapar_lta'}, inplace=True)

## HCWI

In [9]:
# Load Data
hcwi = pd.read_csv('C:/Users/spiterisr/OneDrive - centralbankmalta.org/Working Papers/Beat the Heat Hackathon/Data/Climate Data/Heat Intensity/intensity_2001-2022.csv')
hcwi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113730 entries, 0 to 113729
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   date     113730 non-null  object 
 1   year     113730 non-null  int64  
 2   NUTS_ID  113730 non-null  object 
 3   median   113730 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.5+ MB


In [10]:
# Transform Dataset to Monthly

hcwi['date'] = pd.to_datetime(hcwi['date'])
hcwi['month'] = hcwi['date'].dt.month

dfs = []
for i in hcwi['year'].unique():
    for j in hcwi['month'].unique():
        df = pd.DataFrame()
        df['NUTS_ID'] = hcwi['NUTS_ID'].unique()
        df['year'] = i
        df['month'] = j
        dfs.append(df)
hcwi_base = pd.concat(dfs)

hcwi = pd.merge(hcwi_base, hcwi.groupby(['NUTS_ID', 'year', 'month']).agg({'median':'median'}), on=['NUTS_ID', 'year', 'month'])

In [11]:
# Descriptive Statistics Format

hcwi_aggregated = hcwi.copy()
hcwi_aggregated.rename(columns={'median': 'hcwi'}, inplace=True)

# calculate standard deviation per year
additional_variable = hcwi_aggregated.groupby(['NUTS_ID', 'year']).agg({'hcwi':'std'})
additional_variable = additional_variable.add_suffix('_std')
hcwi_aggregated = pd.merge(hcwi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate mean per year
additional_variable = hcwi_aggregated.groupby(['NUTS_ID', 'year']).agg({'hcwi':'mean'})
additional_variable = additional_variable.add_suffix('_mean')
hcwi_aggregated = pd.merge(hcwi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate max per year
additional_variable = hcwi_aggregated.groupby(['NUTS_ID', 'year']).agg({'hcwi':'max'})
additional_variable = additional_variable.add_suffix('_max')
hcwi_aggregated = pd.merge(hcwi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# lets create yearly data. Just use any of  max or min function as it is already calculated for yearly data
f = {'hcwi_std': 'min',
     'hcwi_mean': 'min',
     'hcwi_max': 'min'}
hcwi_aggregated = hcwi_aggregated.groupby(['NUTS_ID', 'year'], as_index=False).agg(f)

hcwi_aggregated.head()

NUTS_ID  year  hcwi_std  hcwi_mean  hcwi_max
0   AL011  2001       NaN   1.340070  1.340070
1   AL011  2003  0.377647   1.654242  1.921278
2   AL011  2004       NaN   0.812006  0.812006
3   AL011  2007  0.816290   2.643816  3.663824
4   AL011  2008       NaN   1.057826  1.057826

In [12]:
# MIDAS Format

counter = 1
for i in hcwi['month'].unique():
    if counter == 1:
        m_df = hcwi[hcwi['month'] == i]
        m_df = m_df[['NUTS_ID', 'year', 'median']]
        m_df.rename(columns={'median': 'HCWI ['+str(i)+']'}, inplace=True)
    else:
        m2_df = hcwi[hcwi['month'] == i]
        m2_df = m2_df[['NUTS_ID', 'year', 'median']]
        m2_df.rename(columns={'median': 'HCWI ['+str(i)+']'}, inplace=True)
        m_df = pd.merge(m_df, m2_df, on=['NUTS_ID', 'year'], how='outer')
    counter += 1

hcwi_midas = m_df.copy()
hcwi_midas.head()

NUTS_ID  year  HCWI [1]  HCWI [2]  HCWI [3]  HCWI [4]  HCWI [5]  HCWI [6]  \
0   HU331  2001  0.699417  3.120047  1.266849       NaN       NaN       NaN   
1   HU233  2001  0.468749  3.430002  1.594792       NaN       NaN       NaN   
2   HR024  2001  2.423235  1.313408       NaN       NaN       NaN       NaN   
3   HR033  2001  1.596077       NaN       NaN       NaN       NaN       NaN   
4   HR034  2001  1.592936       NaN       NaN       NaN       NaN       NaN   

   HCWI [7]  HCWI [8]  HCWI [9]  HCWI [10]  HCWI [11]  HCWI [12]  
0       NaN       NaN       NaN   1.769317        NaN        NaN  
1       NaN       NaN       NaN   2.066420        NaN        NaN  
2       NaN       NaN       NaN   0.915208        NaN        NaN  
3       NaN       NaN       NaN        NaN        NaN        NaN  
4       NaN       NaN       NaN        NaN        NaN        NaN

In [13]:
# Add LTAs

LTA = getLTA(hcwi, 'median', np.mean, '_LTA')
hcwi_aggregated = pd.merge(hcwi_aggregated, LTA, on=['NUTS_ID'])
hcwi_aggregated.rename(columns={'median_LTA': 'hcwi_lta'}, inplace=True)
hcwi_midas = pd.merge(hcwi_midas, LTA, on=['NUTS_ID'])
hcwi_midas.rename(columns={'median_LTA': 'hcwi_lta'}, inplace=True)

## LFI

In [14]:
# Get Files in LFI Folder
location = "C:/Users/spiterisr/OneDrive - centralbankmalta.org/Working Papers/Beat the Heat Hackathon/Data/Climate Data/Low Flow/"
all_files = os.listdir(location)

In [15]:
# Store Files as 1 Dataframe

all_dfs = []

for file in all_files:
    if file.endswith("csv"):
        file_name = file.split("/")[-1]
        year = file_name.split("_")[-1].split(".")[0]
        
        lfi_yearly = pd.read_csv(location+file)
        lfi_yearly["year"] = year
        # column names has whitespaces around
        lfi_yearly.rename(columns=lambda x: x.strip(), inplace=True)
        all_dfs.append(lfi_yearly)
        
lfi = pd.concat(all_dfs)
lfi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337656 entries, 0 to 15347
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   NUTS_ID      337656 non-null  object 
 1   TIMESTAMP    337656 non-null  object 
 2   MONTHLY_MAX  337656 non-null  float64
 3   year         337656 non-null  object 
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


In [16]:
# Descriptive Statistics Format

lfi_aggregated = lfi.copy()
lfi_aggregated.rename(columns={'MONTHLY_MAX': 'lfi'}, inplace=True)

# calculate standard deviation per year
additional_variable = lfi_aggregated.groupby(['NUTS_ID', 'year']).agg({'lfi':'std'})
additional_variable = additional_variable.add_suffix('_std')
lfi_aggregated = pd.merge(lfi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate mean per year
additional_variable = lfi_aggregated.groupby(['NUTS_ID', 'year']).agg({'lfi':'mean'})
additional_variable = additional_variable.add_suffix('_mean')
lfi_aggregated = pd.merge(lfi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate max per year
additional_variable = lfi_aggregated.groupby(['NUTS_ID', 'year']).agg({'lfi':'max'})
additional_variable = additional_variable.add_suffix('_max')
lfi_aggregated = pd.merge(lfi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# lets create yearly data. Just use any of  max or min function as it is already calculated for yearly data
f = {'lfi_std': 'min',
     'lfi_mean': 'min',
     'lfi_max': 'min'}
lfi_aggregated = lfi_aggregated.groupby(['NUTS_ID', 'year'], as_index=False).agg(f)

lfi_aggregated.head()

NUTS_ID  year   lfi_std  lfi_mean  lfi_max
0   AL011  2001  0.384583  0.248333    0.889
1   AL011  2002  0.320707  0.163000    0.893
2   AL011  2003  0.315661  0.181000    0.888
3   AL011  2004  0.000000  0.000000    0.000
4   AL011  2005  0.190716  0.076667    0.614

In [17]:
# MIDAS Format

lfi['TIMESTAMP'] = pd.to_datetime(lfi['TIMESTAMP'])
lfi['month'] = lfi['TIMESTAMP'].dt.month

counter = 1
for i in lfi['month'].unique():
    if counter == 1:
        m_df = lfi[lfi['month'] == i]
        m_df = m_df[['NUTS_ID', 'year', 'MONTHLY_MAX']]
        m_df.rename(columns={'MONTHLY_MAX': 'LFI ['+str(i)+']'}, inplace=True)
    else:
        m2_df = lfi[lfi['month'] == i]
        m2_df = m2_df[['NUTS_ID', 'year', 'MONTHLY_MAX']]
        m2_df.rename(columns={'MONTHLY_MAX': 'LFI ['+str(i)+']'}, inplace=True)
        m_df = pd.merge(m_df, m2_df, on=['NUTS_ID', 'year'], how='outer')
    counter += 1

lfi_midas = m_df.copy()
lfi_midas.head()

NUTS_ID  year  LFI [1]  LFI [2]  LFI [3]  LFI [4]  LFI [5]  LFI [6]  \
0   AL011  2001      0.0    0.000      0.0    0.420      0.0      0.0   
1   AL013  2001      0.0    0.000      0.0    0.485      0.0      0.0   
2   AL014  2001      0.0    0.000      0.0    0.000      0.0      0.0   
3   AL015  2001      0.0    0.000      0.0    0.000      0.0      0.0   
4   AL021  2001      0.0    0.579      0.0    0.273      0.0      0.0   

   LFI [7]  LFI [8]  LFI [9]  LFI [10]  LFI [11]  LFI [12]  
0      0.0    0.000      0.0     0.790     0.881     0.889  
1      0.0    0.000      0.0     0.790     0.881     0.948  
2      0.0    0.000      0.0     0.000     0.000     0.000  
3      0.0    0.158      0.0     0.761     0.846     0.947  
4      0.0    0.000      0.0     0.452     0.708     0.895

In [18]:
# Add LTAs

LTA = getLTA(lfi, 'MONTHLY_MAX', np.mean, '_LTA')
lfi_aggregated = pd.merge(lfi_aggregated, LTA, on=['NUTS_ID'])
lfi_aggregated.rename(columns={'MONTHLY_MAX_LTA': 'lfi_lta'}, inplace=True)
lfi_midas = pd.merge(lfi_midas, LTA, on=['NUTS_ID'])
lfi_midas.rename(columns={'MONTHLY_MAX_LTA': 'lfi_lta'}, inplace=True)

## SMA

In [19]:
# Get Files in SMA Folder
location = "C:/Users/spiterisr/OneDrive - centralbankmalta.org/Working Papers/Beat the Heat Hackathon/Data/Climate Data/SMA/"
all_files = os.listdir(location)

In [20]:
# Store Files as 1 Dataframe

all_dfs = []

for file in all_files:
    if file.endswith("csv"):
        file_name = file.split("/")[-1]
        year = file_name.split("_")[-1].split(".")[0]
        
        sma_yearly = pd.read_csv(location+file)
        sma_yearly["year"] = year
        # column names has whitespaces around
        sma_yearly.rename(columns=lambda x: x.strip(), inplace=True)
        all_dfs.append(sma_yearly)
        
sma = pd.concat(all_dfs)
sma.dropna(subset=['MEDIAN'], inplace=True)
sma.rename(columns={'NUTS_CODE': 'NUTS_ID'}, inplace=True)
sma.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395472 entries, 0 to 17975
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   NUTS_ID    395472 non-null  object 
 1   TIMESTAMP  395472 non-null  object 
 2   MEDIAN     395472 non-null  float64
 3   year       395472 non-null  object 
dtypes: float64(1), object(3)
memory usage: 15.1+ MB


In [21]:
# Descriptive Statistics Format

sma_aggregated = sma.copy()
sma_aggregated.rename(columns={'MEDIAN': 'sma'}, inplace=True)

# calculate standard deviation per year
additional_variable = sma_aggregated.groupby(['NUTS_ID', 'year']).agg({'sma':'std'})
additional_variable = additional_variable.add_suffix('_std')
sma_aggregated = pd.merge(sma_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate mean per year
additional_variable = sma_aggregated.groupby(['NUTS_ID', 'year']).agg({'sma':'mean'})
additional_variable = additional_variable.add_suffix('_mean')
sma_aggregated = pd.merge(sma_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate number of low months
SMA_combined_count = sma_aggregated.copy() 
SMA_combined_count['MEDIAN'] = np.where(SMA_combined_count['sma'] < -1, 1, 0)
additional_variable = SMA_combined_count.groupby(['year','NUTS_ID']).agg({'sma':'sum'})
additional_variable = additional_variable.add_suffix('_count')
sma_aggregated = pd.merge(sma_aggregated,additional_variable,on=['year','NUTS_ID'])

# calculate max per year
additional_variable = sma_aggregated.groupby(['NUTS_ID', 'year']).agg({'sma':'max'})
additional_variable = additional_variable.add_suffix('_max')
sma_aggregated = pd.merge(sma_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate min per year
additional_variable = sma_aggregated.groupby(['NUTS_ID', 'year']).agg({'sma':'min'})
additional_variable = additional_variable.add_suffix('_min')
sma_aggregated = pd.merge(sma_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# lets create yearly data. Just use any of  max or min function as it is already calculated for yearly data
f = {'sma_std': 'min',
     'sma_mean': 'min',
     'sma_max': 'min',
     'sma_min': 'min', 
     'sma_count': 'min'}
sma_aggregated = sma_aggregated.groupby(['NUTS_ID', 'year'], as_index=False).agg(f)

sma_aggregated.head()

NUTS_ID  year   sma_std  sma_mean  sma_max  sma_min  sma_count
0      AD  2001  0.869501 -0.284667    0.876   -1.373     -3.416
1      AD  2002  0.929988  0.009250    1.344   -1.353      0.111
2      AD  2003  0.733818  0.514000    1.362   -0.807      6.168
3      AD  2004  1.054231 -0.220917    1.089   -1.785     -2.651
4      AD  2005  1.305523 -1.206750    0.349   -3.087    -14.481

In [22]:
# MIDAS Format

sma['TIMESTAMP'] = pd.to_datetime(sma['TIMESTAMP'])
sma['month'] = sma['TIMESTAMP'].dt.month

counter = 1
for i in sma['month'].unique():
    if counter == 1:
        m_df = sma[sma['month'] == i]
        m_df = m_df[['NUTS_ID', 'year', 'MEDIAN']]
        m_df.rename(columns={'MEDIAN': 'SMA ['+str(i)+']'}, inplace=True)
    else:
        m2_df = sma[sma['month'] == i]
        m2_df = m2_df[['NUTS_ID', 'year', 'MEDIAN']]
        m2_df.rename(columns={'MEDIAN': 'SMA ['+str(i)+']'}, inplace=True)
        m_df = pd.merge(m_df, m2_df, on=['NUTS_ID', 'year'], how='outer')
    counter += 1

sma_midas = m_df.copy()
sma_midas.head()

NUTS_ID  year  SMA [1]  SMA [2]  SMA [3]  SMA [4]  SMA [5]  SMA [6]  \
0      AD  2001    0.815    0.786    0.876    0.461    0.226   -0.855   
1   AL011  2001   -0.324   -0.453    0.086    0.255   -0.089   -0.070   
2   AL012  2001    0.735    0.343    0.684    0.921    0.304   -0.062   
3   AL013  2001    0.573    0.338    0.475    0.757    0.315    0.182   
4   AL014  2001    0.785    0.558    0.737    0.988    0.597   -0.046   

   SMA [7]  SMA [8]  SMA [9]  SMA [10]  SMA [11]  SMA [12]  
0   -0.207   -0.925   -1.090    -1.017    -1.373    -1.113  
1   -0.166   -0.264    0.110    -0.692    -0.993    -0.681  
2   -0.320   -0.604    0.944    -0.522    -0.254    -0.253  
3   -0.420   -0.955    0.579    -0.184    -0.153    -0.081  
4   -0.469   -0.825    0.629    -0.313    -0.289    -0.398

In [23]:
# Add LTAs

LTA = getLTA(sma, 'MEDIAN', np.mean, '_LTA')
sma_aggregated = pd.merge(sma_aggregated, LTA, on=['NUTS_ID'])
sma_aggregated.rename(columns={'MEDIAN_LTA': 'sma_lta'}, inplace=True)
sma_midas = pd.merge(sma_midas, LTA, on=['NUTS_ID'])
sma_midas.rename(columns={'MEDIAN_LTA': 'sma_lta'}, inplace=True)

## SPI

In [24]:
# Get Files in SPI Folder
location = "C:/Users/spiterisr/OneDrive - centralbankmalta.org/Working Papers/Beat the Heat Hackathon/Data/Climate Data/SPI/"
all_files = os.listdir(location)

In [25]:
# Store Files as 1 Dataframe

all_dfs = []

for file in all_files:
    if file.endswith("csv"):
        file_name = file.split("/")[-1]
        spa_type = file_name.split("_")[0]
        year = file_name.split("_")[-1].split(".")[0]
        df_a1 = pd.read_csv(location+file)
        df_a1.rename(columns=lambda x: x.strip(), inplace=True)
        df_a1['year'] = pd.to_datetime(df_a1['TIMESTAMP']).dt.year
        df_a1.rename(columns={'MEDIAN': f'MEDIAN_{spa_type}'}, inplace=True)
        all_dfs.append(df_a1)
    
spi = pd.concat(all_dfs, ignore_index=True)
spi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1577473 entries, 0 to 1577472
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   NUTS_ID       1577473 non-null  object 
 1   TIMESTAMP     1577473 non-null  object 
 2   MEDIAN_spa01  398893 non-null   float64
 3   year          1577473 non-null  int64  
 4   MEDIAN_spa03  398904 non-null   float64
 5   MEDIAN_spa06  398904 non-null   float64
 6   MEDIAN_spa12  380772 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 84.2+ MB


In [26]:
# Transform Dataset to Monthly

spi['TIMESTAMP'] = pd.to_datetime(spi['TIMESTAMP'])
spi['month'] = spi['TIMESTAMP'].dt.month

dfs = []
for i in spi['year'].unique():
    for j in spi['month'].unique():
        df = pd.DataFrame()
        df['NUTS_ID'] = spi['NUTS_ID'].unique()
        df['year'] = i
        df['month'] = j
        dfs.append(df)
spi_base = pd.concat(dfs)

spi = pd.merge(spi_base, spi.groupby(['NUTS_ID', 'year', 'month']).agg({'MEDIAN_spa01':'median', 'MEDIAN_spa12':'median'}), on=['NUTS_ID', 'year', 'month'])
spi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398904 entries, 0 to 398903
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   NUTS_ID       398904 non-null  object 
 1   year          398904 non-null  int64  
 2   month         398904 non-null  int64  
 3   MEDIAN_spa01  398893 non-null  float64
 4   MEDIAN_spa12  380772 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 18.3+ MB


In [27]:
# Descriptive Statistics Format

spi_aggregated = spi.copy()
spi_aggregated.rename(columns={'MEDIAN_spa01': 'spi01', 'MEDIAN_spa12': 'spi12'}, inplace=True)

# calculate standard deviation per year

additional_variable = spi_aggregated.groupby(['NUTS_ID', 'year']).agg({'spi01':'std'})
additional_variable = additional_variable.add_suffix('_std')
spi_aggregated = pd.merge(spi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

additional_variable = spi_aggregated.groupby(['NUTS_ID', 'year']).agg({'spi12':'std'})
additional_variable = additional_variable.add_suffix('_std')
spi_aggregated = pd.merge(spi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate mean per year
additional_variable = spi_aggregated.groupby(['NUTS_ID', 'year']).agg({'spi01':'mean'})
additional_variable = additional_variable.add_suffix('_mean')
spi_aggregated = pd.merge(spi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

additional_variable = spi_aggregated.groupby(['NUTS_ID', 'year']).agg({'spi12':'mean'})
additional_variable = additional_variable.add_suffix('_mean')
spi_aggregated = pd.merge(spi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate max per year
additional_variable = spi_aggregated.groupby(['NUTS_ID', 'year']).agg({'spi01':'max'})
additional_variable = additional_variable.add_suffix('_max')
spi_aggregated = pd.merge(spi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

additional_variable = spi_aggregated.groupby(['NUTS_ID', 'year']).agg({'spi12':'max'})
additional_variable = additional_variable.add_suffix('_max')
spi_aggregated = pd.merge(spi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate min per year
additional_variable = spi_aggregated.groupby(['NUTS_ID', 'year']).agg({'spi01':'min'})
additional_variable = additional_variable.add_suffix('_min')
spi_aggregated = pd.merge(spi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

additional_variable = spi_aggregated.groupby(['NUTS_ID', 'year']).agg({'spi12':'min'})
additional_variable = additional_variable.add_suffix('_min')
spi_aggregated = pd.merge(spi_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# lets create yearly data. Just use any of  max or min function as it is already calculated for yearly data
f = {'spi01_std': 'min',
     'spi01_mean': 'min',
     'spi01_max': 'min',
     'spi01_min': 'min', 
     'spi12_std': 'min',
     'spi12_mean': 'min',
     'spi12_max': 'min',
     'spi12_min': 'min',}
spi_aggregated = spi_aggregated.groupby(['NUTS_ID', 'year'], as_index=False).agg(f)

spi_aggregated.head()

NUTS_ID  year  spi01_std  spi01_mean  spi01_max  spi01_min  spi12_std  \
0   AL011  2001   0.637692    0.035167      0.978     -1.490   0.341675   
1   AL011  2002   1.053242    0.109000      1.994     -1.600   0.577491   
2   AL011  2003   1.220293   -0.370917      2.063     -2.370   0.631263   
3   AL011  2004   0.535808    0.599667      1.412     -0.113   0.658651   
4   AL011  2005   0.661310    0.359083      1.439     -0.880   0.384765   

   spi12_mean  spi12_max  spi12_min  
0   -0.631667     -0.109     -1.090  
1   -0.625750      0.253     -1.280  
2    0.229250      1.105     -1.060  
3    1.171667      1.820     -0.301  
4    0.492667      1.023      0.051

In [28]:
# MIDAS Format

counter = 1
for i in spi['month'].unique():
    if counter == 1:
        m_df = spi[spi['month'] == i]
        m_df = m_df[['NUTS_ID', 'year', 'MEDIAN_spa01']]
        m_df.rename(columns={'MEDIAN_spa01': 'SPI01 ['+str(i)+']'}, inplace=True)
    else:
        m2_df = spi[spi['month'] == i]
        m2_df = m2_df[['NUTS_ID', 'year', 'MEDIAN_spa01']]
        m2_df.rename(columns={'MEDIAN_spa01': 'SPI01 ['+str(i)+']'}, inplace=True)
        m_df = pd.merge(m_df, m2_df, on=['NUTS_ID', 'year'], how='outer')
    counter += 1
    
for i in spi['month'].unique():
    m2_df = spi[spi['month'] == i]
    m2_df = m2_df[['NUTS_ID', 'year', 'MEDIAN_spa12']]
    m2_df.rename(columns={'MEDIAN_spa12': 'SPI12 ['+str(i)+']'}, inplace=True)
    m_df = pd.merge(m_df, m2_df, on=['NUTS_ID', 'year'], how='outer')

spi_midas = m_df.copy()
spi_midas.head()

NUTS_ID  year  SPI01 [1]  SPI01 [2]  SPI01 [3]  SPI01 [4]  SPI01 [5]  \
0   AL011  2001     0.1130      0.147      0.087     0.8100     0.2020   
1   AL012  2001     0.2905      0.145      0.234     0.8985     0.4100   
2   AL013  2001     0.3270      0.510      0.072     1.1030     0.1555   
3   AL014  2001     0.3320      0.395      0.318     0.8620    -0.1440   
4   AL015  2001     0.5720      0.297      0.404     0.9250    -0.1900   

   SPI01 [6]  SPI01 [7]  SPI01 [8]  ...  SPI12 [3]  SPI12 [4]  SPI12 [5]  \
0    -0.2750    -0.1890     -0.413  ...    -0.9630     -1.060    -0.6800   
1    -0.4445    -0.3995     -0.913  ...    -0.5345     -0.414    -0.1140   
2     0.1810     0.2995     -0.164  ...    -0.4855     -0.234     0.0465   
3     0.1000    -0.1110     -0.498  ...    -0.1940     -0.147     0.0450   
4     0.4240    -0.1020     -0.328  ...     0.2210      0.454     0.6690   

   SPI12 [6]  SPI12 [7]  SPI12 [8]  SPI12 [9]  SPI12 [10]  SPI12 [11]  \
0    -0.5910    -0.5680    -0.4590     -0.109      -0.388     -0.4650   
1    -0.0415    -0.2870    -0.2645      0.128      -0.019     -0.0665   
2     0.1205     0.1475     0.3400      0.361       0.112      0.2060   
3     0.2060     0.0850     0.1180      0.203       0.029      0.0560   
4     0.7020     0.5720     0.6250      0.706       0.444      0.3650   

   SPI12 [12]  
0     -0.1670  
1     -0.2265  
2      0.2895  
3      0.0640  
4      0.2570  

[5 rows x 26 columns]

In [29]:
# Add LTAs

LTA01 = getLTA(spi, 'MEDIAN_spa01', np.mean, '_LTA')
LTA12 = getLTA(spi, 'MEDIAN_spa12', np.mean, '_LTA')

spi_aggregated = pd.merge(spi_aggregated, LTA01, on=['NUTS_ID'])
spi_aggregated = pd.merge(spi_aggregated, LTA12, on=['NUTS_ID'])
spi_aggregated.rename(columns={'MEDIAN_spa01_LTA': 'spa01_lta', 'MEDIAN_spa12_LTA': 'spa12_lta'}, inplace=True)

spi_midas = pd.merge(spi_midas, LTA01, on=['NUTS_ID'])
spi_midas = pd.merge(spi_midas, LTA12, on=['NUTS_ID'])
spi_midas.rename(columns={'MEDIAN_spa01_LTA': 'spa01_lta', 'MEDIAN_spa12_LTA': 'spa12_lta'}, inplace=True)

## Maximum Temperature

In [30]:
# Get Files in Maximum Temperature Folder
location = "C:/Users/spiterisr/OneDrive - centralbankmalta.org/Working Papers/Beat the Heat Hackathon/Data/Climate Data/Maximum Temperature/"
all_files = os.listdir(location)

In [31]:
# Store Files as 1 Dataframe

all_dfs = []

for file in all_files:
    if file.endswith("csv"):
        file_name = file.split("/")[-1]
        year = file_name.split("_")[-1].split(".")[0]
        df_a1 = pd.read_csv(location+file)
        df_a1['year'] = pd.to_datetime(df_a1['TIMESTAMP']).dt.year
        all_dfs.append(df_a1)
        
max_temp = pd.concat(all_dfs)
max_temp = max_temp[max_temp['MEDIAN'].notnull()]
max_temp = max_temp.rename(columns={'MEDIAN': 'MaxTemp', 'NUTS_CODE':'NUTS_ID'})
max_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 358734 entries, 0 to 13009
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   NUTS_ID    358734 non-null  object 
 1   TIMESTAMP  358734 non-null  object 
 2   MaxTemp    358734 non-null  float64
 3   year       358734 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 13.7+ MB


In [32]:
# Descriptive Statistics Format

max_temp_aggregated = max_temp.copy()

# calculate standard deviation per year
additional_variable = max_temp_aggregated.groupby(['NUTS_ID', 'year']).agg({'MaxTemp':'std'})
additional_variable = additional_variable.add_suffix('_std')
max_temp_aggregated = pd.merge(max_temp_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate mean per year
additional_variable = max_temp_aggregated.groupby(['NUTS_ID', 'year']).agg({'MaxTemp':'mean'})
additional_variable = additional_variable.add_suffix('_mean')
max_temp_aggregated = pd.merge(max_temp_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate max per year
additional_variable = max_temp_aggregated.groupby(['NUTS_ID', 'year']).agg({'MaxTemp':'max'})
additional_variable = additional_variable.add_suffix('_max')
max_temp_aggregated = pd.merge(max_temp_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# calculate min per year
additional_variable = max_temp_aggregated.groupby(['NUTS_ID', 'year']).agg({'MaxTemp':'min'})
additional_variable = additional_variable.add_suffix('_min')
max_temp_aggregated = pd.merge(max_temp_aggregated,additional_variable,on=['NUTS_ID', 'year'])

# lets create yearly data. Just use any of  max or min function as it is already calculated for yearly data
f = {'MaxTemp_std': 'min',
     'MaxTemp_mean': 'min',
     'MaxTemp_max': 'min',
     'MaxTemp_min': 'min'}
max_temp_aggregated = max_temp_aggregated.groupby(['NUTS_ID', 'year'], as_index=False).agg(f)

max_temp_aggregated.head()

NUTS_ID  year  MaxTemp_std  MaxTemp_mean  MaxTemp_max  MaxTemp_min
0   AL011  2001     8.688941     15.288750       27.933        0.070
1   AL011  2002     7.426680     14.994833       25.711        3.397
2   AL011  2003     9.245145     15.456167       28.508        3.044
3   AL011  2004     7.816177     14.754667       25.564        1.644
4   AL011  2005     8.268229     13.819500       25.197        2.852

In [33]:
# MIDAS Format

max_temp['TIMESTAMP'] = pd.to_datetime(max_temp['TIMESTAMP'])
max_temp['month'] = max_temp['TIMESTAMP'].dt.month

counter = 1
for i in max_temp['month'].unique():
    if counter == 1:
        m_df = max_temp[max_temp['month'] == i]
        m_df = m_df[['NUTS_ID', 'year', 'MaxTemp']]
        m_df.rename(columns={'MaxTemp': 'MaxTemp ['+str(i)+']'}, inplace=True)
    else:
        m2_df = max_temp[max_temp['month'] == i]
        m2_df = m2_df[['NUTS_ID', 'year', 'MaxTemp']]
        m2_df.rename(columns={'MaxTemp': 'MaxTemp ['+str(i)+']'}, inplace=True)
        m_df = pd.merge(m_df, m2_df, on=['NUTS_ID', 'year'], how='outer')
    counter += 1

max_temp_midas = m_df.copy()
max_temp_midas.head()

NUTS_ID  year  MaxTemp [1]  MaxTemp [2]  MaxTemp [3]  MaxTemp [4]  \
0   AL011  2001        5.730        6.480       12.897       13.449   
1   AL012  2001       13.260       12.896       19.479       18.974   
2   AL013  2001        3.682        5.399       11.549       12.462   
3   AL014  2001        9.742       10.075       16.634       17.164   
4   AL015  2001        7.825        8.006       14.571       15.492   

   MaxTemp [5]  MaxTemp [6]  MaxTemp [7]  MaxTemp [8]  MaxTemp [9]  \
0       19.454       22.320       26.497       27.933       20.997   
1       25.658       27.598       32.045       33.200       26.825   
2       18.512       21.471       24.802       26.525       19.381   
3       23.117       25.184       29.810       31.042       24.142   
4       21.173       23.173       28.200       29.250       22.050   

   MaxTemp [10]  MaxTemp [11]  MaxTemp [12]  
0        17.794         9.844         0.070  
1        23.575        15.759         7.492  
2        17.117         8.075        -1.398  
3        21.628        13.184         4.283  
4        19.886        11.645         3.234

In [34]:
# Add LTAs

LTA = getLTA(max_temp, 'MaxTemp', np.mean, '_lta')
max_temp_aggregated = pd.merge(max_temp_aggregated, LTA, on=['NUTS_ID'])
max_temp_midas = pd.merge(max_temp_midas, LTA, on=['NUTS_ID'])

## Combine as 1 Dataset & Save

In [35]:
# Combine Aggregated Data

max_temp_aggregated['year'] = max_temp_aggregated['year'].astype(str)
spi_aggregated['year'] = spi_aggregated['year'].astype(str)
sma_aggregated['year'] = sma_aggregated['year'].astype(str)
lfi_aggregated['year'] = lfi_aggregated['year'].astype(str)
hcwi_aggregated['year'] = hcwi_aggregated['year'].astype(str)
fapar_aggregated['year'] = fapar_aggregated['year'].astype(str)

aggregated = pd.merge(max_temp_aggregated, spi_aggregated, on=['year', 'NUTS_ID'], how='outer')
aggregated = pd.merge(aggregated, sma_aggregated, on=['year', 'NUTS_ID'], how='outer')
aggregated = pd.merge(aggregated, lfi_aggregated, on=['year', 'NUTS_ID'], how='outer')
aggregated = pd.merge(aggregated, hcwi_aggregated, on=['year', 'NUTS_ID'], how='outer')
aggregated = pd.merge(aggregated, fapar_aggregated, on=['year', 'NUTS_ID'], how='outer')
aggregated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34653 entries, 0 to 34652
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NUTS_ID           34653 non-null  object 
 1   year              34653 non-null  object 
 2   MaxTemp_std       30123 non-null  float64
 3   MaxTemp_mean      30123 non-null  float64
 4   MaxTemp_max       30123 non-null  float64
 5   MaxTemp_min       30123 non-null  float64
 6   MaxTemp_lta       30123 non-null  float64
 7   spi01_std         33242 non-null  float64
 8   spi01_mean        33242 non-null  float64
 9   spi01_max         33242 non-null  float64
 10  spi01_min         33242 non-null  float64
 11  spi12_std         31731 non-null  float64
 12  spi12_mean        31731 non-null  float64
 13  spi12_max         31731 non-null  float64
 14  spi12_min         31731 non-null  float64
 15  spa01_lta         33242 non-null  float64
 16  spa12_lta         33242 non-null  float6

In [36]:
# Combine MIDAS Data

max_temp_midas['year'] = max_temp_midas['year'].astype(str)
spi_midas['year'] = spi_midas['year'].astype(str)
sma_midas['year'] = sma_midas['year'].astype(str)
lfi_midas['year'] = lfi_midas['year'].astype(str)
hcwi_midas['year'] = hcwi_midas['year'].astype(str)
fapar_midas['year'] = fapar_midas['year'].astype(str)

midas = pd.merge(max_temp_midas, spi_midas, on=['year', 'NUTS_ID'], how='outer')
midas = pd.merge(midas, sma_midas, on=['year', 'NUTS_ID'], how='outer')
midas = pd.merge(midas, lfi_midas, on=['year', 'NUTS_ID'], how='outer')
midas = pd.merge(midas, hcwi_midas, on=['year', 'NUTS_ID'], how='outer')
midas = pd.merge(midas, fapar_midas, on=['year', 'NUTS_ID'], how='outer')
midas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34653 entries, 0 to 34652
Data columns (total 93 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NUTS_ID       34653 non-null  object 
 1   year          34653 non-null  object 
 2   MaxTemp [1]   30110 non-null  float64
 3   MaxTemp [2]   30110 non-null  float64
 4   MaxTemp [3]   30113 non-null  float64
 5   MaxTemp [4]   30118 non-null  float64
 6   MaxTemp [5]   30116 non-null  float64
 7   MaxTemp [6]   30114 non-null  float64
 8   MaxTemp [7]   30117 non-null  float64
 9   MaxTemp [8]   30118 non-null  float64
 10  MaxTemp [9]   30102 non-null  float64
 11  MaxTemp [10]  30116 non-null  float64
 12  MaxTemp [11]  28800 non-null  float64
 13  MaxTemp [12]  28800 non-null  float64
 14  MaxTemp_lta   30123 non-null  float64
 15  SPI01 [1]     33242 non-null  float64
 16  SPI01 [2]     33242 non-null  float64
 17  SPI01 [3]     33242 non-null  float64
 18  SPI01 [4]     33242 non-nu

In [37]:
# Save Combined Datasets
aggregated.to_csv('climate_aggregated_variables.csv', index=False)
midas.to_csv('climate_midas_variables.csv', index=False)